 # Lab 3: 품질 평가 - 충실도



 번역 결과의 의미 보존, 정확성을 평가합니다.



 ## 학습 목표



 - 번역 충실도(Faithfulness) 평가 방법 이해

 - Claude를 평가자(Judge)로 활용

 - 0-5 점수 체계 적용 및 구조화된 평가 결과 생성



 ## 워크샵 전체 흐름



 ```

 Lab 1: 환경 설정 ✅

 ↓

 Lab 2: 번역 기법 비교 ✅

 ↓

 Lab 3: 품질 평가 - 충실도 (현재)

 ↓

 Lab 4: 품질 평가 - 용어 일관성

 ↓

 Lab 5: 품질 평가 - 문화/톤

 ↓

 Lab 6: 피드백 기반 재번역

 ↓

 Lab 7: 종합 분석

 ```

 ---

 ## 3.1 환경 설정 및 이전 Lab 결과 로드

In [2]:
import json
import os
from datetime import datetime

import boto3
import pandas as pd


In [3]:
# 설정
REGION = "us-west-2"
OUTPUT_DIR = "lab_outputs"

# Bedrock 클라이언트 생성
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=REGION
)

# 모델 설정
CLAUDE_MODELS = {
    "sonnet": {
        "id": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "name": "Claude Sonnet 4",
    },
    "haiku": {
        "id": "us.anthropic.claude-haiku-4-20250514-v1:0",
        "name": "Claude Haiku 4",
    }
}

DEFAULT_MODEL = "sonnet"
MODEL_ID = CLAUDE_MODELS[DEFAULT_MODEL]["id"]

print(f"Bedrock 클라이언트 초기화 완료")
print(f"사용 모델: {CLAUDE_MODELS[DEFAULT_MODEL]['name']}")


Bedrock 클라이언트 초기화 완료
사용 모델: Claude Sonnet 4


In [4]:
# 유틸리티 함수
def load_results(filename: str) -> dict:
    filepath = os.path.join(OUTPUT_DIR, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"✓ 로드 완료: {filepath}")
    return data

def save_results(data: dict, filename: str) -> str:
    filepath = os.path.join(OUTPUT_DIR, filename)
    data["_metadata"] = {
        "saved_at": datetime.now().isoformat(),
        "model": CLAUDE_MODELS[DEFAULT_MODEL]["name"]
    }
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✓ 저장 완료: {filepath}")
    return filepath

def call_claude(prompt: str, model_id: str = MODEL_ID, max_tokens: int = 2048) -> str:
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = bedrock_client.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body),
        contentType="application/json",
        accept="application/json"
    )
    response_body = json.loads(response["body"].read())
    return response_body["content"][0]["text"]


In [5]:
# Lab 2 결과 로드
zero_shot_data = load_results("translations_zero_shot.json")
few_shot_data = load_results("translations_few_shot.json")
glossary_data = load_results("translations_glossary.json")

print(f"\n로드된 번역 결과:")
print(f"  - Zero-shot: {len(zero_shot_data['translations'])}건")
print(f"  - Few-shot: {len(few_shot_data['translations'])}건")
print(f"  - 용어집: {len(glossary_data['translations'])}건")


✓ 로드 완료: lab_outputs/translations_zero_shot.json
✓ 로드 완료: lab_outputs/translations_few_shot.json
✓ 로드 완료: lab_outputs/translations_glossary.json

로드된 번역 결과:
  - Zero-shot: 6건
  - Few-shot: 6건
  - 용어집: 6건


 ---

 ## 3.2 충실도 평가란?



 충실도(Faithfulness) 평가는 번역이 원문의 의미를 얼마나 정확하게 전달하는지 측정합니다.



 ### 평가 항목

 - 의미 손실: 원문의 중요한 정보가 빠졌는가?

 - 의미 추가: 원문에 없는 내용이 추가되었는가?

 - 의미 왜곡: 원문과 다른 의미로 번역되었는가?

 - 숫자/단위 정확성: 숫자, 날짜, 단위가 정확한가?



 ### 점수 체계 (0-5)



 | 점수 | 판정 | 설명 |

 |------|------|------|

 | 5 | Pass | 의미/숫자 완전 일치 |

 | 4 | Pass | 뉘앙스/어순 경미한 차이 |

 | 3 | Review | 핵심 의미 일부 누락/추가 (검수 필요) |

 | 0-2 | Fail | 사실 오류/단위 오염 |

 ---

 ## 3.3 충실도 평가 프롬프트 설계

In [6]:
FAITHFULNESS_EVAL_PROMPT = """당신은 번역 품질 평가 전문가입니다.
아래 원문과 번역문을 비교하여 충실도(Faithfulness)를 평가해주세요.

## 원문 (한국어)
{source}

## 번역문 (영어)
{translation}

## 평가 기준
- 의미 손실: 원문의 중요한 정보가 빠졌는가?
- 의미 추가: 원문에 없는 내용이 추가되었는가?
- 의미 왜곡: 원문과 다른 의미로 번역되었는가?
- 숫자/단위 정확성: 숫자, 날짜, 단위가 정확한가?

## 점수 기준
- 5점: 의미/숫자 완전 일치
- 4점: 뉘앙스/어순 경미한 차이
- 3점: 핵심 의미 일부 누락/추가 (검수 필요)
- 0-2점: 사실 오류/단위 오염

## 출력 형식
반드시 아래 JSON 형식으로만 응답하세요:
```json
{{
  "score": <0-5 사이 정수>,
  "verdict": "<Pass|Review|Fail>",
  "issues": ["발견된 문제점 1", "발견된 문제점 2"],
  "suggestion": "개선 제안 (문제가 없으면 빈 문자열)",
  "reasoning": "평가 근거를 한국어로 설명"
}}
```"""


 ---

 ## 3.4 평가 함수 정의

In [10]:
def evaluate_faithfulness(source: str, translation: str) -> dict:
    """충실도 평가를 수행하고 결과를 반환합니다."""
    
    prompt = FAITHFULNESS_EVAL_PROMPT.format(
        source=source,
        translation=translation
    )
    
    response = call_claude(prompt)
    
    # JSON 파싱
    try:
        # JSON 블록 추출
        if "```json" in response:
            json_str = response.split("```json")[1].split("```")[0].strip()
        elif "```" in response:
            json_str = response.split("```")[1].split("```")[0].strip()
        else:
            json_str = response.strip()
        
        result = json.loads(json_str)
    except (json.JSONDecodeError, IndexError) as e:
        result = {
            "score": 0,
            "verdict": "Fail",
            "issues": ["평가 응답 파싱 실패"],
            "suggestion": "",
            "reasoning": f"파싱 오류: {str(e)}"
        }
    
    return result


In [11]:
# 평가 테스트
test_source = "삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?"
test_translation = "Do I need a Samsung account to use Samsung Cloud?"

print("평가 테스트")
print(f"원문: {test_source}")
print(f"번역: {test_translation}")
print("-" * 50)

test_result = evaluate_faithfulness(test_source, test_translation)
print(f"점수: {test_result['score']}")
print(f"판정: {test_result['verdict']}")
print(f"근거: {test_result['reasoning']}")


평가 테스트
원문: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?
번역: Do I need a Samsung account to use Samsung Cloud?
--------------------------------------------------
점수: 5
판정: Pass
근거: 원문의 의미가 완벽하게 번역되었습니다. '삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?'라는 질문이 'Do I need a Samsung account to use Samsung Cloud?'로 정확히 번역되어 의미 손실, 추가, 왜곡이 전혀 없습니다. 문법적으로도 자연스럽고 적절한 영어 표현입니다.


 ---

 ## 3.5 세 가지 번역 기법 평가 실행

In [12]:
def run_faithfulness_evaluation(translations: list, method_name: str) -> list:
    """번역 목록에 대해 충실도 평가를 실행합니다."""
    
    print(f"\n{method_name} 충실도 평가 중...")
    print("=" * 80)
    
    results = []
    
    for item in translations:
        eval_result = evaluate_faithfulness(item["source"], item["translation"])
        
        result = {
            "id": item["id"],
            "source": item["source"],
            "translation": item["translation"],
            "method": item["method"],
            "score": eval_result["score"],
            "verdict": eval_result["verdict"],
            "issues": eval_result["issues"],
            "suggestion": eval_result["suggestion"],
            "reasoning": eval_result["reasoning"],
            "eval_type": "faithfulness",
            "timestamp": datetime.now().isoformat()
        }
        results.append(result)
        
        # 결과 출력
        status = "✓" if eval_result["verdict"] == "Pass" else "△" if eval_result["verdict"] == "Review" else "✗"
        print(f"\n[{item['id']}] {status} 점수: {eval_result['score']}/5")
        print(f"  원문: {item['source'][:50]}...")
        print(f"  번역: {item['translation'][:50]}...")
        if eval_result["issues"]:
            print(f"  문제: {', '.join(eval_result['issues'])}")
    
    return results


In [13]:
# Zero-shot 평가
zero_shot_eval = run_faithfulness_evaluation(
    zero_shot_data["translations"], 
    "Zero-shot"
)



Zero-shot 충실도 평가 중...

[IDS_FAQ_SC_MAIN_HEADER_02] ✓ 점수: 5/5
  원문: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?...
  번역: Do I need a Samsung account to use Samsung Cloud?...

[IDS_FAQ_SC_MAIN_HEADER_03] ✓ 점수: 5/5
  원문: 삼성 계정을 단말에서 제거하면 어떻게 되나요?...
  번역: What happens when you remove a Samsung account fro...

[IDS_FAQ_SC_ACCESSING_TEXT] ✓ 점수: 4/5
  원문: 삼성 클라우드 웹페이지 접속 후 Contact us를 통해 '1대1 문의하기'로 문의해주시...
  번역: After accessing the Samsung Cloud webpage, please ...
  문제: 'Contact us'가 중복적으로 언급됨

[IDS_FAQ_SC_MAIN_HEADER_13] ✓ 점수: 5/5
  원문: 갤럭시 단말이 현재 없습니다. 삼성 서비스에 저장 및 동기화했던 데이터나 정보를 확인할 수...
  번역: I don't currently have a Galaxy device. Is there a...

[IDS_FAQ_GO_DEVICE_NOTES_UL_01] ✓ 점수: 5/5
  원문: 특정 단말, 국가, 사업자에 따라 지원 여부는 다를 수 있습니다....
  번역: Support availability may vary depending on the spe...

[IDS_FAQ_GO_MAIN_HEADER_27] ✓ 점수: 5/5
  원문: 어디로 가면 클라우드 휴지통을 확인할 수 있나요?...
  번역: Where can I go to check the cloud recycle bin?...


In [14]:
# Few-shot 평가
few_shot_eval = run_faithfulness_evaluation(
    few_shot_data["translations"], 
    "Few-shot"
)



Few-shot 충실도 평가 중...

[IDS_FAQ_SC_MAIN_HEADER_02] ✓ 점수: 5/5
  원문: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?...
  번역: Do I need a Samsung account to use Samsung Cloud?...

[IDS_FAQ_SC_MAIN_HEADER_03] ✓ 점수: 4/5
  원문: 삼성 계정을 단말에서 제거하면 어떻게 되나요?...
  번역: What happens if I remove my Samsung account from m...
  문제: 'my'라는 소유격이 원문에 명시되지 않았음

[IDS_FAQ_SC_ACCESSING_TEXT] ✓ 점수: 4/5
  원문: 삼성 클라우드 웹페이지 접속 후 Contact us를 통해 '1대1 문의하기'로 문의해주시...
  번역: After accessing the Samsung Cloud webpage, you can...
  문제: 'Contact us'가 중복으로 언급되어 약간의 표현상 어색함이 있음

[IDS_FAQ_SC_MAIN_HEADER_13] ✓ 점수: 5/5
  원문: 갤럭시 단말이 현재 없습니다. 삼성 서비스에 저장 및 동기화했던 데이터나 정보를 확인할 수...
  번역: I don't currently have a Galaxy device. Is there a...

[IDS_FAQ_GO_DEVICE_NOTES_UL_01] ✓ 점수: 5/5
  원문: 특정 단말, 국가, 사업자에 따라 지원 여부는 다를 수 있습니다....
  번역: Support may vary depending on the specific device,...

[IDS_FAQ_GO_MAIN_HEADER_27] ✓ 점수: 4/5
  원문: 어디로 가면 클라우드 휴지통을 확인할 수 있나요?...
  번역: Where can I go to check my Cloud Trash?...
  문제: 'Cloud Trash'보다 'Cloud Recycle Bi

In [15]:
# 용어집 적용 평가
glossary_eval = run_faithfulness_evaluation(
    glossary_data["translations"], 
    "용어집 적용"
)



용어집 적용 충실도 평가 중...

[IDS_FAQ_SC_MAIN_HEADER_02] ✓ 점수: 5/5
  원문: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?...
  번역: Do I need a Samsung account to use Samsung Cloud?...

[IDS_FAQ_SC_MAIN_HEADER_03] ✓ 점수: 4/5
  원문: 삼성 계정을 단말에서 제거하면 어떻게 되나요?...
  번역: What happens when you remove your Samsung account ...
  문제: 'your' 소유격 추가

[IDS_FAQ_SC_ACCESSING_TEXT] ✓ 점수: 4/5
  원문: 삼성 클라우드 웹페이지 접속 후 Contact us를 통해 '1대1 문의하기'로 문의해주시...
  번역: After accessing the Samsung Cloud webpage, please ...
  문제: 원문의 '차단 해제'가 'unblock your account'로 번역되어 'account'라는 단어가 추가됨

[IDS_FAQ_SC_MAIN_HEADER_13] ✓ 점수: 5/5
  원문: 갤럭시 단말이 현재 없습니다. 삼성 서비스에 저장 및 동기화했던 데이터나 정보를 확인할 수...
  번역: I don't currently have a Galaxy device. Is there a...

[IDS_FAQ_GO_DEVICE_NOTES_UL_01] ✓ 점수: 5/5
  원문: 특정 단말, 국가, 사업자에 따라 지원 여부는 다를 수 있습니다....
  번역: Support may vary depending on the specific device,...

[IDS_FAQ_GO_MAIN_HEADER_27] ✓ 점수: 4/5
  원문: 어디로 가면 클라우드 휴지통을 확인할 수 있나요?...
  번역: Where can I go to check the cloud Trash?...
  문제: 'Trash'의 대문자 표기가 부자연스러움

## 3.5.1 치명적 오류 감지 테스트
###  충실도 평가의 핵심 가치는 **치명적 오류를 자동으로 탐지**하는 것입니다.


In [21]:
# 의도적 오류 번역 예시
error_examples = [
    {
        "id": "ERROR_01_숫자오류",
        "source": "RAM 1GB 이상 (갤러리 동기화는 RAM 1.5GB 이상)",
        "translation": "RAM 1TB or more (Gallery sync: RAM 1.5TB or more)",
        "error_type": "숫자/단위 오류 (GB → TB)"
    },
    {
        "id": "ERROR_02_의미반대",
        "source": "삼성 클라우드를 사용하려면 삼성 계정이 필요합니다",
        "translation": "You do not need a Samsung account to use Samsung Cloud",
        "error_type": "의미 반대 (필요 → 불필요)"
    },
    {
        "id": "ERROR_03_정보누락",
        "source": "삼성 클라우드 웹페이지 접속 후 Contact us를 통해 문의해주시면 사용자 정보 확인 후 차단 해제가 가능합니다",
        "translation": "Please contact us through the Samsung Cloud webpage",
        "error_type": "핵심 정보 누락 (차단 해제, 사용자 정보 확인)"
    },
    {
        "id": "ERROR_04_의미왜곡",
        "source": "삼성 계정을 단말에서 제거하면 동기화된 데이터가 삭제됩니다",
        "translation": "If you remove your Samsung account, all your data will be permanently deleted from Samsung servers",
        "error_type": "의미 왜곡 (단말 삭제 → 서버 영구 삭제)"
    }
]

print("=" * 80)
print("치명적 오류 감지 테스트")
print("=" * 80)

for ex in error_examples:
    print(f"\n[{ex['id']}]")
    print(f"오류 유형: {ex['error_type']}")
    print(f"원문: {ex['source']}")
    print(f"오역: {ex['translation']}")

# %%
# 오류 번역 평가 실행
error_eval_results = []

print("\n치명적 오류 평가 중...")
print("=" * 80)

for ex in error_examples:
    eval_result = evaluate_faithfulness(ex["source"], ex["translation"])
    
    result = {
        "id": ex["id"],
        "error_type": ex["error_type"],
        "source": ex["source"],
        "translation": ex["translation"],
        "score": eval_result["score"],
        "verdict": eval_result["verdict"],
        "issues": eval_result["issues"],
        "reasoning": eval_result["reasoning"]
    }
    error_eval_results.append(result)
    
    # 결과 출력
    if eval_result["verdict"] == "Fail":
        status = "✓ 정상 탐지"
    elif eval_result["verdict"] == "Review":
        status = "△ 검수 필요로 분류"
    else:
        status = "✗ 탐지 실패"
    
    print(f"\n[{ex['id']}] {status}")
    print(f"  오류 유형: {ex['error_type']}")
    print(f"  점수: {eval_result['score']}/5 ({eval_result['verdict']})")
    print(f"  탐지 근거: {eval_result['reasoning'][:100]}...")

# %%
# 오류 감지 결과 요약
print("\n" + "=" * 80)
print("치명적 오류 감지 결과 요약")
print("=" * 80)

detected = sum(1 for r in error_eval_results if r["verdict"] in ["Fail", "Review"])
total = len(error_eval_results)

print(f"\n탐지율: {detected}/{total} ({100*detected/total:.0f}%)")
print("\n상세 결과:")
print(f"{'오류 유형':<30} {'점수':<8} {'판정':<10} {'결과':<15}")
print("-" * 70)

for r in error_eval_results:
    if r["verdict"] == "Fail":
        result_str = "✓ 정상 탐지"
    elif r["verdict"] == "Review":
        result_str = "△ 검수 분류"
    else:
        result_str = "✗ 탐지 실패"
    
    print(f"{r['error_type']:<30} {r['score']:<8} {r['verdict']:<10} {result_str:<15}")

print("\n" + "=" * 80)
print("결론: 충실도 평가는 치명적 오류(숫자, 의미 반대, 누락)를 자동으로 탐지하는 안전망 역할")
print("=" * 80)


치명적 오류 감지 테스트

[ERROR_01_숫자오류]
오류 유형: 숫자/단위 오류 (GB → TB)
원문: RAM 1GB 이상 (갤러리 동기화는 RAM 1.5GB 이상)
오역: RAM 1TB or more (Gallery sync: RAM 1.5TB or more)

[ERROR_02_의미반대]
오류 유형: 의미 반대 (필요 → 불필요)
원문: 삼성 클라우드를 사용하려면 삼성 계정이 필요합니다
오역: You do not need a Samsung account to use Samsung Cloud

[ERROR_03_정보누락]
오류 유형: 핵심 정보 누락 (차단 해제, 사용자 정보 확인)
원문: 삼성 클라우드 웹페이지 접속 후 Contact us를 통해 문의해주시면 사용자 정보 확인 후 차단 해제가 가능합니다
오역: Please contact us through the Samsung Cloud webpage

[ERROR_04_의미왜곡]
오류 유형: 의미 왜곡 (단말 삭제 → 서버 영구 삭제)
원문: 삼성 계정을 단말에서 제거하면 동기화된 데이터가 삭제됩니다
오역: If you remove your Samsung account, all your data will be permanently deleted from Samsung servers

치명적 오류 평가 중...

[ERROR_01_숫자오류] ✓ 정상 탐지
  오류 유형: 숫자/단위 오류 (GB → TB)
  점수: 0/5 (Fail)
  탐지 근거: 원문의 'GB(기가바이트)'를 'TB(테라바이트)'로 번역하여 용량이 1000배 잘못 표기되었습니다. 이는 시스템 요구사항에서 치명적인 오류로, 사용자에게 완전히 잘못된 정보를 제...

[ERROR_02_의미반대] ✓ 정상 탐지
  오류 유형: 의미 반대 (필요 → 불필요)
  점수: 0/5 (Fail)
  탐지 근거: 원문은 '삼성 클라우드를 사용하려면 삼성 계정이 필요합니다'로 삼성 계정의 필요성을 명시하고 있으나, 번역문은 'You do not ne

 ---

 ## 3.6 평가 결과 비교

In [16]:
# 점수 요약
def summarize_scores(eval_results: list, method_name: str) -> dict:
    scores = [r["score"] for r in eval_results]
    verdicts = [r["verdict"] for r in eval_results]
    
    return {
        "method": method_name,
        "avg_score": sum(scores) / len(scores),
        "pass_count": verdicts.count("Pass"),
        "review_count": verdicts.count("Review"),
        "fail_count": verdicts.count("Fail"),
        "total": len(scores)
    }

zero_summary = summarize_scores(zero_shot_eval, "Zero-shot")
few_summary = summarize_scores(few_shot_eval, "Few-shot")
glossary_summary = summarize_scores(glossary_eval, "용어집")


In [17]:
# 비교 테이블 출력
print("=" * 80)
print("충실도 평가 결과 비교")
print("=" * 80)

print(f"\n{'기법':<15} {'평균점수':<10} {'Pass':<8} {'Review':<8} {'Fail':<8}")
print("-" * 50)

for summary in [zero_summary, few_summary, glossary_summary]:
    print(f"{summary['method']:<15} {summary['avg_score']:<10.2f} {summary['pass_count']:<8} {summary['review_count']:<8} {summary['fail_count']:<8}")

print("=" * 80)


충실도 평가 결과 비교

기법              평균점수       Pass     Review   Fail    
--------------------------------------------------
Zero-shot       4.83       6        0        0       
Few-shot        4.50       6        0        0       
용어집             4.50       6        0        0       


In [18]:
# 항목별 상세 비교
print("\n항목별 점수 비교")
print("=" * 80)
print(f"{'ID':<35} {'Zero':<8} {'Few':<8} {'Glossary':<8}")
print("-" * 60)

for i in range(len(zero_shot_eval)):
    item_id = zero_shot_eval[i]["id"]
    zero_score = zero_shot_eval[i]["score"]
    few_score = few_shot_eval[i]["score"]
    glossary_score = glossary_eval[i]["score"]
    
    print(f"{item_id:<35} {zero_score:<8} {few_score:<8} {glossary_score:<8}")

print("=" * 80)



항목별 점수 비교
ID                                  Zero     Few      Glossary
------------------------------------------------------------
IDS_FAQ_SC_MAIN_HEADER_02           5        5        5       
IDS_FAQ_SC_MAIN_HEADER_03           5        4        4       
IDS_FAQ_SC_ACCESSING_TEXT           4        4        4       
IDS_FAQ_SC_MAIN_HEADER_13           5        5        5       
IDS_FAQ_GO_DEVICE_NOTES_UL_01       5        5        5       
IDS_FAQ_GO_MAIN_HEADER_27           5        4        4       


 ---

 ## 3.7 평가 결과 저장

In [19]:
# 결과 저장
faithfulness_results = {
    "zero_shot": zero_shot_eval,
    "few_shot": few_shot_eval,
    "glossary": glossary_eval,
    "summary": {
        "zero_shot": zero_summary,
        "few_shot": few_summary,
        "glossary": glossary_summary
    }
}

save_results(faithfulness_results, "eval_faithfulness.json")


✓ 저장 완료: lab_outputs/eval_faithfulness.json


'lab_outputs/eval_faithfulness.json'

 ---

 ## 3.8 Lab 3 완료

In [20]:
print("=" * 80)
print("Lab 3: 품질 평가 - 충실도 완료")
print("=" * 80)

print("\n[평가 결과 요약]")
for summary in [zero_summary, few_summary, glossary_summary]:
    print(f"  - {summary['method']}: 평균 {summary['avg_score']:.2f}점, Pass {summary['pass_count']}건")

print("\n[저장된 파일]")
print(f"  - {OUTPUT_DIR}/eval_faithfulness.json")

print("\n[다음 단계]")
print("  → Lab 4: 품질 평가 - 용어 일관성")
print("    용어집 준수, 브랜드명 정확도를 평가합니다.")

print("\n" + "=" * 80)


Lab 3: 품질 평가 - 충실도 완료

[평가 결과 요약]
  - Zero-shot: 평균 4.83점, Pass 6건
  - Few-shot: 평균 4.50점, Pass 6건
  - 용어집: 평균 4.50점, Pass 6건

[저장된 파일]
  - lab_outputs/eval_faithfulness.json

[다음 단계]
  → Lab 4: 품질 평가 - 용어 일관성
    용어집 준수, 브랜드명 정확도를 평가합니다.

